# Exercise 3:  Cross-validation as resampling techniques, adding more complexity (score 15 points)


### Exercise 3:  Cross-validation as resampling techniques, adding more complexity (score 15 points)

The aim here is to write your own code for another widely popular
resampling technique, the so-called cross-validation method.  Again,
before you start with cross-validation approach, you should scale your
data.

Implement the $k$-fold cross-validation algorithm (write your own
code) and evaluate again the MSE function resulting
from the test folds. You can compare your own code with that from
**Scikit-Learn** if needed. 

Compare the MSE you get from your cross-validation code with the one
you got from your **bootstrap** code. Comment your results. Try $5-10$
folds.  You can also compare your own cross-validation code with the
one provided by **Scikit-Learn**.


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

np.random.seed(4155)

# Generate the data.
nsamples = 100
x = np.random.randn(nsamples)
y = 3*x**2 + np.random.randn(nsamples)


#scale data !!

def cross_val(k_folds: int):
    # TODO: is equal splitting of x and y guaranteed with np.array_split if nsamples%k_folds != 0 ??
    x_fold = np.array_split(ary = x,indices_or_sections = k_folds)
    y_fold = np.array_split(ary = y,indices_or_sections = k_folds)
    
    for k in k_folds:
        xtrain = 
    

cross_val(6)

IndentationError: expected an indented block (<ipython-input-11-8100ecbe69e4>, line 24)

0